In [ ]:
# This function calculates the relevant data for the crossings along a single strand for
# One half the height of the full unlink. It goes from the top of the tangle to the equator
# or the equator to the top for either the original tangle or the mirror image version.
# As parameters, it takes braid, which is the braid word, curr_strand, which keeps track
# of which of the 2b strands the code is on, crossing_num, which keeps track of how
# many unique crossings it's seen so far, crossing_number which associates each of the
# symbols in the braid word with its crossing number (if it has been encountered already),
# orientation, which stores data on whether it's a right-hand or left-hand crossing,
# curr_component, which keeps track of the crossings seen on the current link component,
# flip, which is True if we're moving from bottom to top in the braid word, and False
# if we go top to bottom. Finally, there's flip_orientation, which basically keeps
# track of if we're moving up or down in the glued tangles, regardless of if we're on the mirror image
def parse_braid(braid, curr_strand, crossing_num, crossing_number, orientation, curr_component, flip, flip_orientation):
    if flip: # If we're on the mirror tangle, we want to flip the order we consider the crossings in the braid word
        braid = braid[::-1]
    for i, c in enumerate(braid): # Then go through each element of the braid word and keep track of its index
        if flip: # If we're on the mirror tangle, we have to flip the index to get it to match in the original word
            index = len(braid) - i - 1
            c *= -1 # And we also change from an under crossing to an over crossing
        else: # Otherwise, just set index to be the same since it doesn't change
            index = i
        # The element of the braid word affects the strand we're on if either of these two conditions apply
        if abs(c) == curr_strand or abs(c) == curr_strand + 1:
            if crossing_number[index] == -1: # If we haven't seen this crossing yet, give it a unique number
                crossing_number[index] = crossing_num
                crossing_num += 1
            # Once we know the element affects us, we figure out if we're on the overstrand or understrand
            if c == curr_strand + 1 or (c * -1 == curr_strand): # On the overstrand
                curr_component.append(abs(crossing_number[index])) # Indicate crossing in gauss code
                if c > 0:
                    curr_strand += 1 # If we're on the overstrand and it's positive, we move to the right
                else:
                    curr_strand -= 1 # Otherwise we move to the left
                orientation[crossing_number[index]-1][0] = flip_orientation # Assign relevant orientation information
            else: # Otherwise we're on the understrand
                curr_component.append(abs(crossing_number[index]) * -1) # Indicate crossing in gauss code
                if c > 0:
                    curr_strand -= 1
                    orientation[crossing_number[index] - 1][1] = -1 # understrand to the left
                else:
                    curr_strand += 1
                    orientation[crossing_number[index] - 1][1] = 1  # understrand to the right
    return curr_strand, crossing_num

def make_links(braids, b):
    L1 = make_link(braids[0], braids[1], b)
    L2 = make_link(braids[0], braids[2], b)
    L3 = make_link(braids[1], braids[2], b)
    return L1, L2, L3

# This is, as far as I can tell, the simplest way to determine the orientation of a crossing.
# In the extended gauss code, each cross has +1/-1 associated with it, determined as follows:
# Starting at outgoing direction in the overcrossing, find which way to rotate to turn towards
# The outgoing direction of the undercrossing. Clockwise -> -1, counter-clockwise -> +1.
# There's 8 different crossings, and you can map them to +1/-1 based only on if the over-
# crossing goes up or down, and if the under crossing goes left or right.
def get_orientation(orientation):
    is_up, under = orientation
    if is_up:
        return 1 if under == -1 else -1
    else:
        return under

# Calculates the link associated with a tangle, and the mirror image of another tangle.
def make_link(braid1, braid2, b):
    used_strands = [False] * (2 * b) # Keeps track of which strands haven't been used
    # This will contain two lists, each the same length as the two braid words passed in.
    # It associates a unique number to each crossing (or elemenet in either braid word)
    # so that each time we reach a crossing, we can tell if we have visited it already.
    crossing_number = []
    for braid in [braid1, braid2]:
        # Keep track of the crossing number we assign to each element in the braid word
        # It starts with all elements having an associated crossing number of -1, but these
        # are set to the correct number in parse_braid when they are reached
        crossing_number.append([-1] * len(braid))
    # Contains the gauss code. Starting on an arbitrary strand, keep track of the crossings
    # you go past with a number, positive if you went on the overstrand, and negative if
    # you went on the understrand. Once you loop back to where you started, that's one
    # component of the unlink. Repeat until every strand in the unlink has been fully traversed.
    crossings = []
    curr_component = [] # Crossings encountered in the current component of the unlink
    # orientation will contain [overdir, uc] where overdir is up/down, uc is undercrossing
    orientation = [[False, 0] for i in range(len(braid1) + len(braid2))]
    crossing_num = 1
    # Go along top tangle normally, then mirror tangle in reverse
    # We want to continue this process while some strands in the top tangle haven't been visited.
    # We don't need to keep track of strands in the bottom because it is impossible for all the strands
    # in the mirror tangle to be visited while having unvisited strands in the top tangle.
    while any(not i for i in used_strands):
        curr_strand = -1
        for index, i in enumerate(used_strands): # Go through all the strands and find one that hasn't been visited
            if not i:
                curr_strand = index
                break
        assert(curr_strand != -1) # Should've updated to new strand
        cn = crossing_num
        while True:
            used_strands[curr_strand] = True # Mark strand as visited
            # Go through the first braid word
            curr_strand, cn = parse_braid(braid1, curr_strand, cn, crossing_number[0], orientation, curr_component, False, False)
            # Then the mirror braid word in reverse
            curr_strand, cn = parse_braid(braid2, curr_strand, cn, crossing_number[1], orientation, curr_component, True, False)
            # Now use the relation to 'cap off' in the mirror tangle
            if curr_strand % 2 == 0:
                curr_strand += 1
            else:
                curr_strand -= 1
            # Now go through the braid words again with the same process, but in the opposite order
            curr_strand, cn = parse_braid(braid2, curr_strand, cn, crossing_number[1], orientation, curr_component, False, True)
            curr_strand, cn = parse_braid(braid1, curr_strand, cn, crossing_number[0], orientation, curr_component, True, True)
            used_strands[curr_strand] = True # Mark new strand in the top tangle as visited
            # Now we have to check if we get back to where we started (and finish the link component)
            if curr_strand % 2 == 0:
                curr_strand += 1
            else:
                curr_strand -= 1
            if used_strands[curr_strand]: # Then we finished the component
                break
            # Otherwise loop through again
        crossing_num = cn
        if len(curr_component) > 0: # Only append if component has crossings (Sage would remove it anyway)
            crossings.append(curr_component[::]) # Make a copy of curr_component
        curr_component = []
    # At this point, we have gone through, visited all the crossings, and marked the orientation of them
    # Now all that's left is to calculate the extended part of the gauss code, as described in get_orientation
    gauss_orientation = []
    for o in orientation:
        gauss_orientation.append(get_orientation(o)) # Get the proper orientation from crossing data
    return Link([crossings, gauss_orientation])

edge_colors = {0: "red", 1: "blue", 2: "green"}
def euler_characteristic(graph):
    # Generate each of the three graphs with one of the tangles missing
    characteristic = len(graph.vertices()) # Number of vertices
    for i in range(3):
        new_graph = Graph()
        new_graph.allow_multiple_edges(True)
        new_graph.add_vertices(range(len(graph.vertices())))
        for edge in graph.edges():
            v1, v2, label = edge
            if label == i:
                continue
            else:
                new_graph.add_edge(v1, v2, label)
        characteristic += new_graph.connected_components_number() # Increment by the number of 2-cells
    return characteristic - len(graph.edges()) # Subtract number of graphs

def make_graph(braids, b):
    permutations = []
    groups = []
    B = BraidGroup(2*b) # Start with the Braid Group with 2b strands
    for i in range(3):
        braid_group = B(braids[i]) # Calculate the specific braid group with each word
        groups.append(braid_group)
        permutations.append(braid_group.permutation()) # And then generate the permutation
        # The permutation shows where the top of each strand ends up. Therefore,
        # the vertices permutation[0] and permutation[1] are the endpoints of one strand
        # in the tangle, as are permutation[2], permutation[3], etc.
        # It calculates the permutation in top-to-bottom order, so that
        # the permutation of [1,2] on 3 strands is [3,1,2]

    graph = Graph()
    graph.allow_multiple_edges(True)
    graph.add_vertices(range(2*b)) # create 2b vertices in the graph

    for i in range(0, 2*b, 2): # Go from 0 to 2b stepping by 2
        first_strand = i
        second_strand = i + 1
        for j in range(3):
            vertex1 = permutations[j][first_strand] - 1 # permutation is 1, ..., 2b, we want 0, ..., 2b-1
            vertex2 = permutations[j][second_strand] - 1
            graph.add_edge(vertex1, vertex2, j) # Add an edge connecting the two vertices; j is label, used for coloring

    plot = graph.graphplot(color_by_label=edge_colors) # Color the edges by the label (which is the tangle the edge came from)
    return graph, plot



def is_orientable(graph):
    if graph.is_bipartite():
        return BipartiteGraph(graph)
    else:
        return None


def orientation_signs(graph, b):
    if graph is None:
        return [1 for i in range(2*b)]
    signs = []
    left = graph.left
    right = graph.right

    for vertex in graph.vertices():
        if vertex in left:
            signs.append(1)
        else:
            signs.append(-1)

    return signs

def wirtinger_relations(braid, signs, b, prefix):

    B = BraidGroup(2*b)
    braid_elem = B(braid)
    permutation = braid_elem.permutation()

    S = []
    F = FreeGroup(2*b, prefix)

    for i in range(2*b):
        sign = signs[permutation[i]-1]
        S.append(F.gen(i)^sign)

    for i in braid:
        j = abs(i)-1
        if i>0:
            S_jp1 = S[j]
            S_j = S[j]*S[j+1]*S[j]^-1

        if i<0:
            S_j = S[j+1]
            S_jp1 = S[j+1]^-1*S[j]*S[j+1]

        S[j] = S_j
        S[j+1] = S_jp1

    return S, F


'''
Messy code to transform the Tietze representation of a word in the quotient group
ab/bg/ag into the corresponding word in the free group on 6b generators:
(x1, ...., xn, y1, ..., yn, z1, ..., zn)
'''
def get_tietze(word, generators):
    tietze = list(word.Tietze())
    new_tietze = []
    for num in tietze:
        g = str(word.parent().gen(abs(num)-1))
        for index, gen in enumerate(generators):
            if g == str(gen):
                new_tietze.append(-index - 1 if num < 0 else index + 1)
                break
    return new_tietze

'''
Generates the coproduct of two groups quotiented by two lists of words such that
the corresponding words should be equated together.
'''
def coproduct(group1, relations1, group2, relations2, b):
    fg = FreeGroup(group1.gens() + group2.gens())
    quotient = [fg(relations1[i])*fg(relations2[i])^-1 for i in range(len(relations1))]
    # Add the relation for adjacent generators
    quotient = quotient + [fg.gen(2*i)*fg.gen(2*i+1)^-1 for i in range(2*b)]
    coproduct = fg/quotient
    return coproduct

def is_valid(braids, b):
    prefixes = ('x','y','z')
    relations = []
    groups = []
    
    graph, _ = make_graph(braids, b)
    graph = is_orientable(graph)
    signs = orientation_signs(graph, b)

    for i in range(3):
        relation, group = wirtinger_relations(braids[i], signs, b, prefixes[i])
        relations.append(relation)
        groups.append(group)

    ab = coproduct(groups[0], relations[0], groups[1], relations[1], b)
    ab_simp = ab.simplified()
    # -------------------------------------------------------------- #
    ag = coproduct(groups[0], relations[0], groups[2], relations[2], b)
    ag_simp = ag.simplified()
    # -------------------------------------------------------------- #
    bg = coproduct(groups[1], relations[1], groups[2], relations[2], b)
    bg_simp = bg.simplified()
    # print(ab_simp)
    # print(ag_simp)
    # print(bg_simp)
    return all([len(g.relations()) == 0 for g in [ab_simp, ag_simp, bg_simp]])

def second_coproduct(intersection, X1, X1_map, X2, X2_map, fg, offset):
    # fg is freegroup on 8*b generators
    quotient = []
    for gen in intersection.gens():
        # we want w1 to map to the first half of the generators and
        # w2 to shift to the second half of the generators
        w1 = get_tietze(X1_map(gen), list(X1.gens()))
        w2 = get_tietze(X2_map(gen), list(X2.gens()))
        # Shifts generators by positive or negative amount depending on if generator is inverse
        w2 = [i + offset if i > 0 else i - offset for i in w2] # Shift to correct generators
        quotient.append(fg(w1)*fg(w2)^-1)
        
    for rel in X1.relations():
        # Add relationships from the first space
        quotient.append(fg(rel.Tietze()))
        
    for rel in X2.relations():
        fg_rel = rel.Tietze()
        # Add relationships for the second space, but once again shifted to the
        # second half of generators
        fg_rel = [i + offset if i > 0 else i - offset for i in fg_rel]
        quotient.append(fg(fg_rel))
        
    return fg/quotient

def SVK_cube(braids, num_b):
    prefixes = ('x','y','z')
    relations = []
    groups = []

    graph, _ = make_graph(braids, num_b)
    graph = is_orientable(graph)
    signs = orientation_signs(graph, num_b)

    for i in range(3):
        # relations show homomorphism from equatorial S^2 to tangle complement
        relation, group = wirtinger_relations(braids[i], signs, num_b, prefixes[i])
        relations.append(relation)
        groups.append(group)
        
    a, b, g = groups

    ab = coproduct(a, relations[0], b, relations[1], num_b)
    X_1 = ab.simplification_isomorphism() # \pi_1(X_1)
    ag = coproduct(a, relations[0], g, relations[2], num_b)
    X_3 = ag.simplification_isomorphism() # \pi_1(X_3)
    bg = coproduct(b, relations[1], g, relations[2], num_b)
    X_2 = bg.simplification_isomorphism() # \pi_1(X_2)
        
    a_fg = FreeGroup(8*num_b, 'a')
    b_fg = FreeGroup(8*num_b, 'b')
    g_fg = FreeGroup(8*num_b, 'g')
        
    final_answer1 = second_coproduct(a, ab, X_1, ag, X_3, a_fg, 4*num_b)
    final_answer2 = second_coproduct(g, bg, X_2, ag, X_3, g_fg, 4*num_b)
    final_answer3 = second_coproduct(b, ab, X_1, bg, X_2, b_fg, 4*num_b)

    return [final_answer1, final_answer2, final_answer3]

def alexander_matrix(f_group, t):
    return f_group.alexander_matrix([t for i in f_group.gens()])


In [ ]:
Q.<t> = LaurentPolynomialRing(ZZ, 1)
def alexander_invariant(braid, n):
    g1, g2, g3 = SVK_cube(braid, n)
    invariants = []
    for g in [g1, g2, g3]:
        a = alexander_matrix(g.simplified(), t)
        s = min(a.nrows(), a.ncols()) - 1 if a.nrows() == a.ncols() else min(a.nrows(), a.ncols())
        minors = a.minors(s)
        I = Q.ideal(minors)
        invariants.append(I.groebner_basis())
    return invariants

In [ ]:
import random
index = 0
orientable = 0
sphere = 0
z_fundamental = 0
oriented_sphere = 0
b = 4

while True:
    braids = []
    length = random.randint(2, 30) # Number of crossings
    if index % 500 == 0:
        print(f"{index}")
    index += 1
    for _ in range(3):
        arr = []
        for i in range(length):
            r = random.randint(-2*b + 1, 2*b - 1)
            while r == 0:
                r = random.randint(-2*b + 1, 2*b - 1)
            arr.append(r)
    # braids = [[2,3,4,5,6,7], [], arr]
    # braids = [[2, 3, 4, 5, 6, 7, 3, 4, 5, 6], [2, 3, 4, 5, 6, 7], arr]
        braids.append(arr)
    graph, _ = make_graph(braids, b)
    # if euler_characteristic(graph) != 2:
    #     continue
    # if is_orientable(graph) is None:
    #     continue
    # Lastly, check the gluings
    # print("-------------------------")
    valid = True
    L1, L2, L3 = make_links(braids, b)
    for L in [L1, L2, L3]:
        f_g = L.fundamental_group().simplified()
        # print(f_g)
        if len(f_g.relations()) != 0:
            valid = False
    if valid:
        f = alexander_invariant(braids, b)
        # if len(f.relations()) > 0 or len(f.gens()) > 1:
        print(f)
        print(braids)

In [ ]:
# Spun Trefoil
# braid = [[4,5,6,7,2,3,4,5,6,7],[2,3,4,5,6,7], [2, 3, 4, 5, 6, 7, -7, -7, -7, 1, 1, 1, 3, -5, -4]]
# 2-twist spun trefoil
# braid = [[4,5,6,7,5,6,-2,-2,-2,-3,-2,-3,-2,-3,-2,-3,-2,-3,-2,-3,-2], [4,5,-2,-2,-2],[-2,-2,-2,4,3,5,4,6,7]]

In [ ]:
import  random
# By a theorem of Meier, Zupan, if n <= 3, then any generated knot with be unknotted
n = 4
cnt = 0
while True:
    n = 4
    length = random.randint(5, 30)
    if cnt % 10000 == 0:
        print(cnt)
    cnt += 1
    num_braids = 0
    braid = [[], [], []]
    for i in range(3):
        for _ in range(length):
            x = 0
            while x == 0:
                x = random.randint(-2*n+1, 2*n-1)
            braid[i].append(x)
    if random.randint(0,1):
        braid[0] = [4,5,-2,-2,-2]
        braid[1] = [4,5,6,7,5,6,-2,-2,-2,-3,-2,-3,-2,-3,-2,-3,-2,-3,-2,-3,-2]
    else:
        braid[0] = [4,5,6,7,2,3,4,5,6,7]
        braid[1] = [2,3,4,5,6,7]
    # Now braid contains three arrays satisfying len(braid[i]) == length
    valid = True
    L1, L2, L3 = make_links(braid, n)
    for L in [L1, L2, L3]:
        f_g = L.fundamental_group().simplified()
        # print(f_g)
        if len(f_g.relations()) != 0:
            valid = False
    if not valid:
        continue
    # This is actually three invariants that should all be the same
    invariants = alexander_invariant(braid, n)
    invariant = invariants[0]
    print(invariant)
    print(braid)
        